<a href="https://colab.research.google.com/github/perfectmalcolm/Zawadi-Tunu/blob/main/zawadi_tunu_with_memory_bank_on_adk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get started with Zawadi Tunu Built with Memory Bank on ADK

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/mwanyumba7/Zawadi-Tunu/blob/main/zawadi_tunu_with_memory_bank_on_adk.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
</table>

## Overview
We will be building an Agent `Zawadi Tunu` which is an agent ispecialized in gift recommendation engine designed to elevate the act of gift-giving, transforming it from a chore into a deeply personalized and meaningful experience. Built as a demonstration of the Agent Development Kit (ADK) for Gemini and the Vertex AI Memory Bank, this agent's core strength lies in its ability to retain and recall nuanced information over time.

Instead of generic suggestions, the agent uses its long-term memory to remember key details gleaned from previous conversations: a niece's age and past birthday presents (like the "red bike" in the demo), a spouse's stated preference for aisle seats, or a parent's hint about a desired hobby.

This tutorial demonstrates how to build ADK agents with Memory, including both `InMemoryMemoryService` and `VertexAiMemoryBankService`.

This tutorial will cover:
* Comparing `InMemoryMemoryService` vs. `VertexAiMemoryBankService`.
* Generating memories with ADK and Agent Engine Memory Bank.
* Retrieving memories with ADK and Agent Engine Memory Bank.
* Customizing your Memory Bank instance's behavior.

## Get started

### Install Google Gen AI SDK and other required packages


In [51]:
%pip install google-adk google-cloud-aiplatform --upgrade --quiet

### Authenticate your notebook environment (Colab only)

If you're running this notebook on Google Colab, run the cell below to authenticate your environment.

In [52]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [53]:
# Use the environment variable if the user doesn't provide Project ID.
import os

import vertexai

# fmt: off
PROJECT = "devfest-2025-day-1"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
# fmt: on
if not PROJECT or PROJECT == "[your-project-id]":
    PROJECT = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

client = vertexai.Client(project=PROJECT, location=LOCATION)

# Set environment variables for ADK.
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "TRUE"
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION

## (Optional) Set up logging
To surface ADK logs in your notebook, you may need to configure the logging level of ADK's logger.

In [54]:
import logging
import sys

# Set up a general handler for all logs to be printed in your Colab output.
# We'll set the overall level to INFO.
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    stream=sys.stdout,
)

# Get the specific logger used by ADK.
adk_logger = logging.getLogger("google_adk")

# Set its level to DEBUG to ensure you capture ALL messages from the SDK,
# even if the general level is higher (like INFO).
adk_logger.setLevel(logging.DEBUG)

## Define a simple agent

The ADK Runner defines what session service is used by the agent. The session service is responsible for saving the conversation history.

In this example, we're using the in-memory session service (`InMemorySessionService`). However, you can use any session service with Memory.

If you want to use Google's pre-built Memory tools, you can provide a Memory service to the Runner as well. In this example, we'll start by directly invoking memory so that we can compare different services. So, we're not directly setting `memory_service` in the Runner.

In [55]:
from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai.types import Content, Part

APP_NAME = "memory_example_app"
MODEL = "gemini-2.5-flash"

agent = LlmAgent(
    model=MODEL,
    name="Generic_QA_Agent",
    instruction="Answer the user's questions",
)

session_service = InMemorySessionService()

runner = Runner(agent=agent, app_name=APP_NAME, session_service=session_service)


def call_agent(query, session, user_id):
    content = Content(role="user", parts=[Part(text=query)])
    events = runner.run(user_id=user_id, session_id=session, new_message=content)

    for event in events:
        if event.is_final_response():
            final_response = event.content.parts[0].text
            print("Agent Response: ", final_response)

## Introduction to ADK Memory Service
ADK memory services offer an interface for creating and searching for memories. The [`BaseMemoryService`](https://github.com/google/adk-python/blob/main/src/google/adk/memory/base_memory_service.py) defines the interface for managing this searchable, long-term knowledge store. Its primary responsibilities are:

* Ingesting Information (`add_session_to_memory`): Taking the contents of a (usually completed) Session and adding relevant information to the long-term knowledge store.
* Searching Information (`search_memory`): Allowing an agent (typically via a Tool) to query the knowledge store and retrieve relevant snippets or context based on a search query.

In this colab, we'll cover two options:

* `InMemoryMemoryService`: A in-memory service that allows you to search your conversation history to retrieve the raw dialogue using basic keyword matching.
* `VertexAiMemoryBankService`: A remote service that remotely generates memories for "meaningful" information from user conversation.


You can also interact directly with Memory Bank by using the Agent Engine SDK.

### `InMemoryMemoryService`

[`InMemoryMemoryService`](https://github.com/google/adk-python/blob/main/src/google/adk/memory/in_memory_memory_service.py) persists **all** information in-memory. The events are not condensed, and information is not extracted from them.

In [56]:
from google.adk.memory import InMemoryMemoryService

in_memory_service = InMemoryMemoryService()

### `VertexAiMemoryBankService`
Memory Bank will only persist information that's meaningful for future interactions. [`VertexAiMemoryBankService`](https://github.com/google/adk-python/blob/main/src/google/adk/memory/vertex_ai_memory_bank_service.py) provides a built-in integration between Memory Bank and ADK.

To get started with Memory Bank, you need to first create an Agent Engine. This only takes a few seconds.

If you don't have a Google Cloud project, you can use [Vertex in Express mode, which only requires an API key](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/memory-bank/set-up#authentication).

In [57]:
import vertexai
from google.adk.memory import VertexAiMemoryBankService

client = vertexai.Client(project=PROJECT, location=LOCATION)

agent_engine = client.agent_engines.create(
    config={
        "context_spec": {
            "memory_bank_config": {
                "generation_config": {
                    "model": f"projects/{PROJECT}/locations/{LOCATION}/publishers/google/models/gemini-2.5-flash"
                }
            }
        }
    }
)

memory_bank_service = VertexAiMemoryBankService(
    agent_engine_id=agent_engine.api_resource.name.split("/")[-1],
    project=PROJECT,
    location=LOCATION,
)

INFO:vertexai_genai.agentengines:View progress and logs at https://console.cloud.google.com/logs/query?project=devfest-2025-day-1.
INFO:vertexai_genai.agentengines:Agent Engine created. To use it in another session:
INFO:vertexai_genai.agentengines:agent_engine=client.agent_engines.get(name='projects/568929544199/locations/us-central1/reasoningEngines/7795577023350439936')


## Generating Memories

Memories will be generated from your conversation history stored in the `Session` object. The ADK `add_session_to_memory` method is responsible for triggering memory generation for your memory service, but the exact behavior depends on which service you use:

* `InMemoryMemoryService` persists all information, including the raw dialogue.

* `VertexAiMemoryBankService` only persists *meaningful* information. Not all conversations will result in generated memories. Additionally, the information will be condensed to individual, self-contained memories.

In this example, we create two sessions:
1. `chit_chat_session`, which is a conversation that contains events that may not be meaningful for future interactions.
2. `recommendations_session`, which is a conversation about looking for birthday presents for a family member. It includes implicit information that may be helpful for future conversations.

In [58]:
USER_ID = "My User"

chit_chat_session = await session_service.create_session(
    app_name=APP_NAME, user_id=USER_ID
)

call_agent(
    "what types of questions do you answer", chit_chat_session.id, user_id=USER_ID
)
# Refresh session object.
chit_chat_session = await session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id=chit_chat_session.id
)

INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash, backend: GoogleLLMVariant.VERTEX_AI, stream: False
DEBUG:google_adk.google.adk.models.google_llm:
LLM Request:
-----------------------------------------------------------
System Instruction:
Answer the user's questions

You are an agent. Your internal name is "Generic_QA_Agent".
-----------------------------------------------------------
Config:
{'labels': {'adk_agent_name': 'Generic_QA_Agent'}}
-----------------------------------------------------------
Contents:
{"parts":[{"text":"what types of questions do you answer"}],"role":"user"}
-----------------------------------------------------------
Functions:

-----------------------------------------------------------

INFO:google_adk.google.adk.models.google_llm:Response received from the model.
DEBUG:google_adk.google.adk.models.google_llm:
LLM Response:
-----------------------------------------------------------
Text:
As a large language model,

Agent Response:  As a large language model, I can answer a wide variety of questions across many different topics. You can ask me about:

*   **Facts and Information:** History, science, geography, current events (up to my last update), pop culture, technology, literature, etc.
*   **Explanations:** How things work, concepts, theories, ideas.
*   **Definitions:** What a word or phrase means.
*   **Summaries:** Summarize articles, books, concepts, or documents (if provided).
*   **Creative Writing:** Generate stories, poems, scripts, song lyrics, or other creative text.
*   **Problem Solving:** Help with math problems, logical puzzles, or general reasoning questions.
*   **Language Tasks:** Translation (to some extent), grammar correction, rephrasing, generating text in different styles.
*   **General Advice/Suggestions:** Offer general recommendations or ideas on various non-personal topics.
*   **Coding Assistance:** Explain code, debug simple issues, or generate code snippets in vari

In [59]:
recommendations_session = await session_service.create_session(
    app_name=APP_NAME, user_id=USER_ID
)

call_agent(
    "I have a three year old niece. What recommendations do you have for birthday presents?",
    recommendations_session.id,
    user_id=USER_ID,
)
call_agent(
    "I like the idea of a bike. Are there any age appropriate options?",
    recommendations_session.id,
    user_id=USER_ID,
)

recommendations_session = await session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id=recommendations_session.id
)

INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash, backend: GoogleLLMVariant.VERTEX_AI, stream: False
DEBUG:google_adk.google.adk.models.google_llm:
LLM Request:
-----------------------------------------------------------
System Instruction:
Answer the user's questions

You are an agent. Your internal name is "Generic_QA_Agent".
-----------------------------------------------------------
Config:
{'labels': {'adk_agent_name': 'Generic_QA_Agent'}}
-----------------------------------------------------------
Contents:
{"parts":[{"text":"I have a three year old niece. What recommendations do you have for birthday presents?"}],"role":"user"}
-----------------------------------------------------------
Functions:

-----------------------------------------------------------

INFO:google_adk.google.adk.models.google_llm:Response received from the model.
DEBUG:google_adk.google.adk.models.google_llm:
LLM Response:
-------------------------------------------

Agent Response:  That's a fun age to shop for! Three-year-olds are really developing their imagination, fine motor skills, and curiosity. Here are some recommendations, keeping different types of play and development in mind:

**1. Imaginative & Pretend Play:**
*   **Play Kitchen or Food Set:** A classic for a reason! Kids love to "cook" and serve.
*   **Doctor's Kit:** Encourages empathy and helps demystify doctor visits.
*   **Dress-Up Clothes:** Princess outfits, superhero capes, animal costumes, or even just fun hats and scarves can spark hours of play.
*   **Dollhouse & Figures:** Simple, sturdy dollhouses with a few family figures can lead to wonderful storytelling.
*   **Toy Animals/Figures:** A collection of farm animals, dinosaurs, or community helpers for creating their own worlds.

**2. Creative & Fine Motor Skills:**
*   **Large Building Blocks (e.g., DUPLO, Mega Bloks):** Perfect for small hands to build towers, houses, and anything they can imagine.
*   **Play-Doh or Mode

DEBUG:google_adk.google.adk.models.google_llm:
LLM Request:
-----------------------------------------------------------
System Instruction:
Answer the user's questions

You are an agent. Your internal name is "Generic_QA_Agent".
-----------------------------------------------------------
Config:
{'labels': {'adk_agent_name': 'Generic_QA_Agent'}}
-----------------------------------------------------------
Contents:
{"parts":[{"text":"I have a three year old niece. What recommendations do you have for birthday presents?"}],"role":"user"}
{"parts":[{"text":"That's a fun age to shop for! Three-year-olds are really developing their imagination, fine motor skills, and curiosity. Here are some recommendations, keeping different types of play and development in mind:\n\n**1. Imaginative & Pretend Play:**\n*   **Play Kitchen or Food Set:** A classic for a reason! Kids love to \"cook\" and serve.\n*   **Doctor's Kit:** Encourages empathy and helps demystify doctor visits.\n*   **Dress-Up Clothes

Agent Response:  Yes, absolutely! A bike is a fantastic idea for a three-year-old, as it encourages outdoor play, develops gross motor skills, balance, and coordination. At this age, you have a couple of excellent, age-appropriate options:

1.  **Balance Bikes:**
    *   **What they are:** These bikes have no pedals or training wheels. Kids push off the ground with their feet to propel themselves forward and learn to balance by coasting.
    *   **Why they're great for 3-year-olds:** This is often the *best* option for this age. A 3-year-old can quickly learn to glide on a balance bike, developing a natural sense of balance and steering. When they eventually transition to a pedal bike, they often skip training wheels entirely because they've already mastered balance.
    *   **Benefits:** Promotes confidence, develops coordination, and makes learning to ride a pedal bike much easier in the future.
    *   **Things to look for:**
        *   **Lightweight:** Easier for them to maneuver 

### InMemoryMemoryService

First, let's see what memories are extracted from the conversation using `InMemoryMemoryService`

The memories include the raw dialogue of the conversation.

In [60]:
await in_memory_service.add_session_to_memory(chit_chat_session)
await in_memory_service.add_session_to_memory(recommendations_session)

In [61]:
# Peek at the memories.
in_memory_service._session_events

{'memory_example_app/My User': {'64be8ba1-a0a7-4911-ac32-32f84f356b70': [Event(content=Content(
     parts=[
       Part(
         text='what types of questions do you answer'
       ),
     ],
     role='user'
   ), grounding_metadata=None, partial=None, turn_complete=None, finish_reason=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=None, live_session_resumption_update=None, input_transcription=None, output_transcription=None, avg_logprobs=None, logprobs_result=None, cache_metadata=None, citation_metadata=None, invocation_id='e-2443715e-a694-4357-b2f6-b4486aa13c29', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}, requested_tool_confirmations={}, compaction=None, end_of_agent=None, agent_state=None, rewind_before_invocation_id=None), long_running_tool_ids=None, branch=None, id='5ba29b33-548a-4bad-b5cc-267841d8e71f', timest

In [62]:
await in_memory_service.search_memory(app_name=APP_NAME, user_id=USER_ID, query="test")

SearchMemoryResponse(memories=[])

In [63]:
await in_memory_service.search_memory(
    app_name=APP_NAME,
    user_id=USER_ID,
    query="what should I get my mom for mother's day",
)

SearchMemoryResponse(memories=[MemoryEntry(content=Content(
  parts=[
    Part(
      text='what types of questions do you answer'
    ),
  ],
  role='user'
), author='user', timestamp='2025-10-31T10:24:53.830161'), MemoryEntry(content=Content(
  parts=[
    Part(
      text="""As a large language model, I can answer a wide variety of questions across many different topics. You can ask me about:

*   **Facts and Information:** History, science, geography, current events (up to my last update), pop culture, technology, literature, etc.
*   **Explanations:** How things work, concepts, theories, ideas.
*   **Definitions:** What a word or phrase means.
*   **Summaries:** Summarize articles, books, concepts, or documents (if provided).
*   **Creative Writing:** Generate stories, poems, scripts, song lyrics, or other creative text.
*   **Problem Solving:** Help with math problems, logical puzzles, or general reasoning questions.
*   **Language Tasks:** Translation (to some extent), grammar c

### Memory Bank via `VertexAiMemoryBankService`

Now, let's use Memory Bank to generate and store memories. Unlike `InMemoryMemoryService`, only content that's meaningful interactions will be persisted. If there's no meaningful content, the `add_session_to_memory` call will be a no-op; memories are not generated.

Memory generation happens in the background. It's a non-blocking operation, so `add_session_to_memory` may complete before memories are actually generated in the background. If you want memory generation to be a blocking operation, use the Agent Engine SDK:

```
client.agent_engines.memories.generate(
    ...,
    config={
        # Default value is True.
        "wait_for_completion": True
    }
)
```

There are two key operations that Memory Bank performs under-the-hood with memory generation:

* **Extraction**: Extracts information about the user from their conversations with the agent. Only information that matches at least one of your instance's memory topics will be persisted.
* **Consolidation**: Identifies if existing memories with the same scope should be deleted or updated based on the extracted information. Memory Bank checks that new memories are not duplicative or contradictory before merging them with existing memories. If existing memories don't overlap with the new information, a new memory will be created.

#### ADK `add_session_to_memory`

In [64]:
await memory_bank_service.add_session_to_memory(chit_chat_session)
await memory_bank_service.add_session_to_memory(recommendations_session)

INFO:google_adk.google.adk.memory.vertex_ai_memory_bank_service:Generate memory response received.
DEBUG:google_adk.google.adk.memory.vertex_ai_memory_bank_service:Generate memory response: name='projects/568929544199/locations/us-central1/reasoningEngines/7795577023350439936/operations/8780758906629521408' metadata={'@type': 'type.googleapis.com/google.cloud.aiplatform.v1beta1.GenerateMemoriesOperationMetadata', 'genericMetadata': {'createTime': '2025-10-31T10:25:51.776861Z', 'updateTime': '2025-10-31T10:25:51.776861Z'}} done=None error=None response=None
INFO:google_adk.google.adk.memory.vertex_ai_memory_bank_service:Generate memory response received.
DEBUG:google_adk.google.adk.memory.vertex_ai_memory_bank_service:Generate memory response: name='projects/568929544199/locations/us-central1/reasoningEngines/7795577023350439936/operations/1759647087558918144' metadata={'@type': 'type.googleapis.com/google.cloud.aiplatform.v1beta1.GenerateMemoriesOperationMetadata', 'genericMetadata': {

If you run this immediately after `add_session_to_memory`, there may be no memories printed. `add_session_to_memory` is a non-blocking function and memory generation can take a few seconds to run.

In [65]:
await memory_bank_service.search_memory(
    app_name=APP_NAME,
    user_id=USER_ID,
    query="What should I get my mom for mother's day",
)

INFO:google_adk.google.adk.memory.vertex_ai_memory_bank_service:Search memory response received.
DEBUG:google_adk.google.adk.memory.vertex_ai_memory_bank_service:Retrieved memory: distance=0.9933563316379599 memory=Memory(
  create_time=datetime.datetime(2025, 10, 31, 10, 25, 54, 376825, tzinfo=TzInfo(UTC)),
  fact='I like the idea of getting a bike as a birthday present for my three-year-old niece.',
  name='projects/568929544199/locations/us-central1/reasoningEngines/7795577023350439936/memories/3144615899823079424',
  scope={
    'app_name': 'memory_example_app',
    'user_id': 'My User'
  },
  update_time=datetime.datetime(2025, 10, 31, 10, 25, 54, 376825, tzinfo=TzInfo(UTC))
)
DEBUG:google_adk.google.adk.memory.vertex_ai_memory_bank_service:Retrieved memory: distance=1.059445197106196 memory=Memory(
  create_time=datetime.datetime(2025, 10, 31, 10, 25, 54, 382024, tzinfo=TzInfo(UTC)),
  fact='I have a three-year-old niece.',
  name='projects/568929544199/locations/us-central1/reas

SearchMemoryResponse(memories=[MemoryEntry(content=Content(
  parts=[
    Part(
      text='I like the idea of getting a bike as a birthday present for my three-year-old niece.'
    ),
  ],
  role='user'
), author='user', timestamp='2025-10-31T10:25:54.376825+00:00'), MemoryEntry(content=Content(
  parts=[
    Part(
      text='I have a three-year-old niece.'
    ),
  ],
  role='user'
), author='user', timestamp='2025-10-31T10:25:54.382024+00:00')])

#### Agent Engine SDK `GenerateMemories`

If I want to interact directly with Memory Bank, I can use the Agent Engine SDK. For example, I can use [`RetrieveMemories`](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/memory-bank/fetch-memories#scope-based) to retrieve my memories with or without similarity search.

Memories are isolated by their `scope`, which is an arbitrary dictionary. `VertexAiMemoryBankService` uses the scope keys `app_name` and `user_id`. `app_name` is set by your Runner, and `user_id` is set when creating your `Session`.

In [66]:
response = client.agent_engines.memories.retrieve(
    name=agent_engine.api_resource.name,
    scope={"app_name": APP_NAME,
           "user_id": USER_ID},
)
list(response)

[RetrieveMemoriesResponseRetrievedMemory(
   memory=Memory(
     create_time=datetime.datetime(2025, 10, 31, 10, 25, 54, 376825, tzinfo=TzInfo(UTC)),
     fact='I like the idea of getting a bike as a birthday present for my three-year-old niece.',
     name='projects/568929544199/locations/us-central1/reasoningEngines/7795577023350439936/memories/3144615899823079424',
     scope={
       'app_name': 'memory_example_app',
       'user_id': 'My User'
     },
     update_time=datetime.datetime(2025, 10, 31, 10, 25, 54, 376825, tzinfo=TzInfo(UTC))
   )
 ),
 RetrieveMemoriesResponseRetrievedMemory(
   memory=Memory(
     create_time=datetime.datetime(2025, 10, 31, 10, 25, 54, 382024, tzinfo=TzInfo(UTC)),
     fact='I have a three-year-old niece.',
     name='projects/568929544199/locations/us-central1/reasoningEngines/7795577023350439936/memories/5450458909036773376',
     scope={
       'app_name': 'memory_example_app',
       'user_id': 'My User'
     },
     update_time=datetime.datetime

You can also use `Retrievememories` to retrieve memories using similarity search.

In [67]:
response = client.agent_engines.memories.retrieve(
    name=agent_engine.api_resource.name,
    scope={"app_name": APP_NAME, "user_id": USER_ID},
    similarity_search_params={
        "search_query": "what should I get my niece for her birthday?"
    },
)
list(response)

[RetrieveMemoriesResponseRetrievedMemory(
   distance=0.7347648884243799,
   memory=Memory(
     create_time=datetime.datetime(2025, 10, 31, 10, 25, 54, 382024, tzinfo=TzInfo(UTC)),
     fact='I have a three-year-old niece.',
     name='projects/568929544199/locations/us-central1/reasoningEngines/7795577023350439936/memories/5450458909036773376',
     scope={
       'app_name': 'memory_example_app',
       'user_id': 'My User'
     },
     update_time=datetime.datetime(2025, 10, 31, 10, 25, 54, 382024, tzinfo=TzInfo(UTC))
   )
 ),
 RetrieveMemoriesResponseRetrievedMemory(
   distance=0.7547651547113708,
   memory=Memory(
     create_time=datetime.datetime(2025, 10, 31, 10, 25, 54, 376825, tzinfo=TzInfo(UTC)),
     fact='I like the idea of getting a bike as a birthday present for my three-year-old niece.',
     name='projects/568929544199/locations/us-central1/reasoningEngines/7795577023350439936/memories/3144615899823079424',
     scope={
       'app_name': 'memory_example_app',
      

Memory Bank uses a process called "consolidation" to ensure that we do not upload duplicative or contradictory information for the same scope.

In [68]:
import pprint

operation = client.agent_engines.memories.generate(
    name=agent_engine.api_resource.name,
    scope={"app_name": APP_NAME, "user_id": USER_ID},
    direct_contents_source={
        "events": [
            {
                "content": {
                    "role": "user",
                    "parts": [
                        {
                            "text": "I like the idea of getting my niece a bike for her third birthday"
                        }
                    ],
                }
            }
        ]
    },
)
print("***OPERATION RESPONSE***")
pprint.pprint(operation)

print("\n***GENERATED MEMORIES***")
for generated_memory in operation.response.generated_memories:
    pprint.pprint(client.agent_engines.memories.get(name=generated_memory.memory.name))

***OPERATION RESPONSE***
AgentEngineGenerateMemoriesOperation(
  done=True,
  metadata={
    '@type': 'type.googleapis.com/google.cloud.aiplatform.v1beta1.GenerateMemoriesOperationMetadata',
    'genericMetadata': {
      'createTime': '2025-10-31T10:26:05.171373Z',
      'updateTime': '2025-10-31T10:26:10.423233Z'
    }
  },
  name='projects/568929544199/locations/us-central1/reasoningEngines/7795577023350439936/operations/1692093093148360704',
  response=GenerateMemoriesResponse(
    generated_memories=[
      GenerateMemoriesResponseGeneratedMemory(
        action=<GenerateMemoriesResponseGeneratedMemoryAction.UPDATED: 'UPDATED'>,
        memory=Memory(
          name='projects/568929544199/locations/us-central1/reasoningEngines/7795577023350439936/memories/3144615899823079424'
        )
      ),
    ]
  )
)

***GENERATED MEMORIES***
Memory(
  create_time=datetime.datetime(2025, 10, 31, 10, 25, 54, 376825, tzinfo=TzInfo(UTC)),
  fact='I like the idea of getting my niece a bike for h

What if your agent extracted memories that you want to combine with your existing memories? You can also use `GenerateMemories` to consolidate pre-extracted facts for the same scope [documenation](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/memory-bank/generate-memories#consolidate-pre-extracted-memories).

Memory Bank already contains memories on this topic. So, the relevant existing memories will be updated rather than creating a new, duplicative memory.

In [69]:
operation = client.agent_engines.memories.generate(
    name=agent_engine.api_resource.name,
    scope={"app_name": APP_NAME, "user_id": USER_ID},
    direct_memories_source={"direct_memories": [{"fact": "I got my niece a red bike"}]},
)

print("***OPERATION RESPONSE***")
pprint.pprint(operation)

print("\n***GENERATED MEMORIES***")
for generated_memory in operation.response.generated_memories:
    if generated_memory.action != "DELETED":
        pprint.pprint(
            client.agent_engines.memories.get(name=generated_memory.memory.name)
        )

***OPERATION RESPONSE***
AgentEngineGenerateMemoriesOperation(
  done=True,
  metadata={
    '@type': 'type.googleapis.com/google.cloud.aiplatform.v1beta1.GenerateMemoriesOperationMetadata',
    'genericMetadata': {
      'createTime': '2025-10-31T10:26:16.353898Z',
      'updateTime': '2025-10-31T10:26:25.340280Z'
    }
  },
  name='projects/568929544199/locations/us-central1/reasoningEngines/7795577023350439936/operations/8730093410821603328',
  response=GenerateMemoriesResponse(
    generated_memories=[
      GenerateMemoriesResponseGeneratedMemory(
        action=<GenerateMemoriesResponseGeneratedMemoryAction.UPDATED: 'UPDATED'>,
        memory=Memory(
          name='projects/568929544199/locations/us-central1/reasoningEngines/7795577023350439936/memories/3144615899823079424'
        )
      ),
    ]
  )
)

***GENERATED MEMORIES***
Memory(
  create_time=datetime.datetime(2025, 10, 31, 10, 25, 54, 376825, tzinfo=TzInfo(UTC)),
  fact='I got my niece a red bike for her third birthday

#### Automate with Callbacks

To automate the triggering of memory generation, you can implement a callback to generate memories after every turn. See the [`Remember this: Agent state and memory with ADK` blog post](https://cloud.google.com/blog/topics/developers-practitioners/remember-this-agent-state-and-memory-with-adk) for more information.

In [70]:
from google import adk


async def auto_save_session_to_memory_callback(callback_context):
    # Use the invocation context to access the conversation history that should
    # be used as the data source for memory generation.
    await memory_bank_service.add_session_to_memory(
        callback_context._invocation_context.session
    )
    print("\n****Triggered memory generation****\n")


agent = LlmAgent(
    model=MODEL,
    name="Generic_QA_Agent",
    instruction="Answer the user's questions",
    tools=[adk.tools.preload_memory_tool.PreloadMemoryTool()],
    after_agent_callback=auto_save_session_to_memory_callback,
)

runner = Runner(
    agent=agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_bank_service,
)

In [71]:
session = await session_service.create_session(app_name=APP_NAME, user_id=USER_ID)

call_agent(
    "What christmas present recommendations do you have for my niece?",
    session.id,
    user_id=USER_ID,
)

call_agent(
    "I think I'll get her a doll. What types are good for her age?",
    session.id,
    user_id=USER_ID,
)

INFO:google_adk.google.adk.memory.vertex_ai_memory_bank_service:Search memory response received.
DEBUG:google_adk.google.adk.memory.vertex_ai_memory_bank_service:Retrieved memory: distance=0.8091518668940462 memory=Memory(
  create_time=datetime.datetime(2025, 10, 31, 10, 25, 54, 382024, tzinfo=TzInfo(UTC)),
  fact='I have a three-year-old niece.',
  name='projects/568929544199/locations/us-central1/reasoningEngines/7795577023350439936/memories/5450458909036773376',
  scope={
    'app_name': 'memory_example_app',
    'user_id': 'My User'
  },
  update_time=datetime.datetime(2025, 10, 31, 10, 25, 54, 382024, tzinfo=TzInfo(UTC))
)
DEBUG:google_adk.google.adk.memory.vertex_ai_memory_bank_service:Retrieved memory: distance=0.8911007396912826 memory=Memory(
  create_time=datetime.datetime(2025, 10, 31, 10, 25, 54, 376825, tzinfo=TzInfo(UTC)),
  fact='I got my niece a red bike for her third birthday.',
  name='projects/568929544199/locations/us-central1/reasoningEngines/7795577023350439936/m

Agent Response:  Based on your previous messages, your niece is three years old. Since she just received a red bike for her birthday, here are some Christmas present recommendations that she might enjoy and that offer different types of play:

1.  **Building Blocks (LEGO DUPLO or Wooden Blocks):** Great for developing fine motor skills, creativity, and problem-solving. A large set of DUPLO bricks or colorful wooden blocks can provide hours of open-ended play.
2.  **Play Kitchen or Tool Bench:** Encourages imaginative role-play and helps develop social skills. She can "cook" meals or "fix" things, mimicking adults.
3.  **Art Supplies:** A large easel with washable paints, chunky crayons, or a big roll of paper can spark creativity. Look for toddler-friendly, non-toxic options.
4.  **Picture Books:** Three-year-olds love engaging stories with colorful illustrations. Interactive books with flaps or textures are also a hit. You could get a few new titles to expand her library.
5.  **Puzzle

INFO:google_adk.google.adk.memory.vertex_ai_memory_bank_service:Generate memory response received.
DEBUG:google_adk.google.adk.memory.vertex_ai_memory_bank_service:Generate memory response: name='projects/568929544199/locations/us-central1/reasoningEngines/7795577023350439936/operations/5559559273152774144' metadata={'@type': 'type.googleapis.com/google.cloud.aiplatform.v1beta1.GenerateMemoriesOperationMetadata', 'genericMetadata': {'createTime': '2025-10-31T10:26:40.561284Z', 'updateTime': '2025-10-31T10:26:40.561284Z'}} done=None error=None response=None



****Triggered memory generation****



INFO:google_adk.google.adk.memory.vertex_ai_memory_bank_service:Search memory response received.
DEBUG:google_adk.google.adk.memory.vertex_ai_memory_bank_service:Retrieved memory: distance=0.9376008297421816 memory=Memory(
  create_time=datetime.datetime(2025, 10, 31, 10, 25, 54, 382024, tzinfo=TzInfo(UTC)),
  fact='I have a three-year-old niece.',
  name='projects/568929544199/locations/us-central1/reasoningEngines/7795577023350439936/memories/5450458909036773376',
  scope={
    'app_name': 'memory_example_app',
    'user_id': 'My User'
  },
  update_time=datetime.datetime(2025, 10, 31, 10, 25, 54, 382024, tzinfo=TzInfo(UTC))
)
DEBUG:google_adk.google.adk.memory.vertex_ai_memory_bank_service:Retrieved memory: distance=0.9477699117276959 memory=Memory(
  create_time=datetime.datetime(2025, 10, 31, 10, 25, 54, 376825, tzinfo=TzInfo(UTC)),
  fact='I got my niece a red bike for her third birthday.',
  name='projects/568929544199/locations/us-central1/reasoningEngines/7795577023350439936/m

Agent Response:  That's a lovely idea! Dolls are fantastic for fostering nurturing play, imagination, and social-emotional development in three-year-olds.

Here are some types of dolls that are generally great for her age:

1.  **Soft-Bodied Baby Dolls:**
    *   **Why they're great:** These are often the first type of doll kids connect with. They're soft, huggable, easy for little hands to carry, and perfect for role-playing as a parent (feeding, rocking, changing diapers). Many come with simple outfits that are easy to put on and take off, which is great for developing fine motor skills.
    *   **Look for:** Dolls with soft cloth bodies, sometimes with vinyl heads, hands, and feet. Washable options are a bonus! Brands like Baby Stella, Corolle Mon Premier, or similar generic soft baby dolls are excellent.

2.  **Toddler/Pre-schooler Dolls (e.g., "My First" or "Bitty Baby" style):**
    *   **Why they're great:** These dolls are often designed to look like slightly older children rat

INFO:google_adk.google.adk.memory.vertex_ai_memory_bank_service:Generate memory response received.
DEBUG:google_adk.google.adk.memory.vertex_ai_memory_bank_service:Generate memory response: name='projects/568929544199/locations/us-central1/reasoningEngines/7795577023350439936/operations/655139278946304000' metadata={'@type': 'type.googleapis.com/google.cloud.aiplatform.v1beta1.GenerateMemoriesOperationMetadata', 'genericMetadata': {'createTime': '2025-10-31T10:26:48.557364Z', 'updateTime': '2025-10-31T10:26:48.557364Z'}} done=None error=None response=None



****Triggered memory generation****



In [72]:
response = client.agent_engines.memories.retrieve(
    name=agent_engine.api_resource.name,
    scope={"app_name": APP_NAME, "user_id": USER_ID},
    similarity_search_params={
        "search_query": "What was I thinking about getting my niece for Christmas?"
    },
)
list(response)

[RetrieveMemoriesResponseRetrievedMemory(
   distance=0.7943347759252403,
   memory=Memory(
     create_time=datetime.datetime(2025, 10, 31, 10, 25, 54, 382024, tzinfo=TzInfo(UTC)),
     fact='I have a three-year-old niece.',
     name='projects/568929544199/locations/us-central1/reasoningEngines/7795577023350439936/memories/5450458909036773376',
     scope={
       'app_name': 'memory_example_app',
       'user_id': 'My User'
     },
     update_time=datetime.datetime(2025, 10, 31, 10, 25, 54, 382024, tzinfo=TzInfo(UTC))
   )
 ),
 RetrieveMemoriesResponseRetrievedMemory(
   distance=0.8010640831147657,
   memory=Memory(
     create_time=datetime.datetime(2025, 10, 31, 10, 25, 54, 376825, tzinfo=TzInfo(UTC)),
     fact='I got my niece a red bike for her third birthday.',
     name='projects/568929544199/locations/us-central1/reasoningEngines/7795577023350439936/memories/3144615899823079424',
     scope={
       'app_name': 'memory_example_app',
       'user_id': 'My User'
     },
     

The prior example used the entire Session to generate memories. However, this can be costly as the size of the session increases.

You can also use callbacks to only save the current turn to Memory Bank. This has the downside, however, where you lose some of the context of the conversation.

In [73]:
async def auto_save_user_turn_to_memory_callback(callback_context):
    last_turn = callback_context._invocation_context.user_content
    client.agent_engines.memories.generate(
        name=agent_engine.api_resource.name,
        scope={"app_name": APP_NAME, "user_id": USER_ID},
        direct_contents_source={"events": [{"content": last_turn}]},
        config={"wait_for_completion": False},
    )
    print("\n****Triggered memory generation****\n")


agent = LlmAgent(
    model=MODEL,
    name="Generic_QA_Agent",
    instruction="Answer the user's questions",
    tools=[adk.tools.preload_memory_tool.PreloadMemoryTool()],
    before_agent_callback=auto_save_user_turn_to_memory_callback,
)

runner = Runner(
    agent=agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_bank_service,
)

In [74]:
call_agent(
    "I think I'll get her a doll. What types are good for her age?",
    session.id,
    user_id=USER_ID,
)


****Triggered memory generation****



INFO:google_adk.google.adk.memory.vertex_ai_memory_bank_service:Search memory response received.
DEBUG:google_adk.google.adk.memory.vertex_ai_memory_bank_service:Retrieved memory: distance=0.9376007979564559 memory=Memory(
  create_time=datetime.datetime(2025, 10, 31, 10, 25, 54, 382024, tzinfo=TzInfo(UTC)),
  fact='I have a three-year-old niece.',
  name='projects/568929544199/locations/us-central1/reasoningEngines/7795577023350439936/memories/5450458909036773376',
  scope={
    'app_name': 'memory_example_app',
    'user_id': 'My User'
  },
  update_time=datetime.datetime(2025, 10, 31, 10, 25, 54, 382024, tzinfo=TzInfo(UTC))
)
DEBUG:google_adk.google.adk.memory.vertex_ai_memory_bank_service:Retrieved memory: distance=0.9477698802830146 memory=Memory(
  create_time=datetime.datetime(2025, 10, 31, 10, 25, 54, 376825, tzinfo=TzInfo(UTC)),
  fact='I got my niece a red bike for her third birthday.',
  name='projects/568929544199/locations/us-central1/reasoningEngines/7795577023350439936/m

Agent Response:  That's a lovely idea! Dolls are fantastic for fostering nurturing play, imagination, and social-emotional development in three-year-olds.

Here are some types of dolls that are generally great for her age:

1.  **Soft-Bodied Baby Dolls:**
    *   **Why they're great:** These are often the first type of doll kids connect with. They're soft, huggable, easy for little hands to carry, and perfect for role-playing as a parent (feeding, rocking, changing diapers). Many come with simple outfits that are easy to put on and take off, which is great for developing fine motor skills.
    *   **Look for:** Dolls with soft cloth bodies, sometimes with vinyl heads, hands, and feet. Washable options are a bonus! Brands like Baby Stella, Corolle Mon Premier, or similar generic soft baby dolls are excellent.

2.  **Toddler/Pre-schooler Dolls (e.g., "My First" or "Bitty Baby" style):**
    *   **Why they're great:** These dolls are often designed to look like slightly older children rat

## Using Memory in your agent

Now, let's link our agent to Memory, so that memories can be fetched by your agent and used for inference. This allows your agent to remember information from prior sessions in a new, empty session.

You can use ADK's built-in tools to fetch memories and incorporate them in the prompt. When using the built-in tools, it's important to provide both a memory tool when creating your Agent and a memory service when defining your Runner.

### `adk.PreloadMemoryTool`
The `PreloadMemoryTool` always retrieves memories at the start of each turn and includes the memories in the system instruction.

Under the hood, it invokes `memory_service.search_memory` for the given User and App Name. Memory is invoked before the LLM is called, so there will not be an associated tool call logged for memory.

We're going to use a [callback](https://google.github.io/adk-docs/callbacks/) to print out the System Instructions. Since we use `before_model_callback`, the callback executes right before the model is called.

<img src="https://google.github.io/adk-docs/assets/callback_flow.png">

In [75]:
from google import adk


def log_system_instructions(callback_context, llm_request):
    """A callback to print the LLM request."""
    print(
        f"\n*System Instruction*:\n{llm_request.config.system_instruction}\n*********\n"
    )


agent = LlmAgent(
    model=MODEL,
    name="Generic_QA_Agent",
    instruction="Answer the user's questions",
    tools=[adk.tools.preload_memory_tool.PreloadMemoryTool()],
    before_model_callback=log_system_instructions,
)

runner = Runner(
    agent=agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_bank_service,
)

This conversation uses a new session, so it doesn't have access to the user's conversation history without using memory.

Since we're using `PreloadMemoryTool`, the retrieved memories will be appended to the System Instructions.

In [76]:
session = await session_service.create_session(app_name=APP_NAME, user_id=USER_ID)

call_agent(
    "Can you remind me what I got my niece for her christmas?",
    session.id,
    user_id=USER_ID,
)

INFO:google_adk.google.adk.memory.vertex_ai_memory_bank_service:Search memory response received.
DEBUG:google_adk.google.adk.memory.vertex_ai_memory_bank_service:Retrieved memory: distance=0.8241405721171347 memory=Memory(
  create_time=datetime.datetime(2025, 10, 31, 10, 25, 54, 376825, tzinfo=TzInfo(UTC)),
  fact='I got my niece a red bike for her third birthday.',
  name='projects/568929544199/locations/us-central1/reasoningEngines/7795577023350439936/memories/3144615899823079424',
  scope={
    'app_name': 'memory_example_app',
    'user_id': 'My User'
  },
  update_time=datetime.datetime(2025, 10, 31, 10, 26, 25, 254356, tzinfo=TzInfo(UTC))
)
DEBUG:google_adk.google.adk.memory.vertex_ai_memory_bank_service:Retrieved memory: distance=0.8461041281672127 memory=Memory(
  create_time=datetime.datetime(2025, 10, 31, 10, 25, 54, 382024, tzinfo=TzInfo(UTC)),
  fact='I have a three-year-old niece.',
  name='projects/568929544199/locations/us-central1/reasoningEngines/7795577023350439936/m


*System Instruction*:
Answer the user's questions

You are an agent. Your internal name is "Generic_QA_Agent".

The following content is from your previous conversations with the user.
They may be useful for answering the user's current query.
<PAST_CONVERSATIONS>
Time: 2025-10-31T10:26:25.254356+00:00
user: I got my niece a red bike for her third birthday.
Time: 2025-10-31T10:25:54.382024+00:00
user: I have a three-year-old niece.
</PAST_CONVERSATIONS>

*********



INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash, backend: GoogleLLMVariant.VERTEX_AI, stream: False
DEBUG:google_adk.google.adk.models.google_llm:
LLM Request:
-----------------------------------------------------------
System Instruction:
Answer the user's questions

You are an agent. Your internal name is "Generic_QA_Agent".

The following content is from your previous conversations with the user.
They may be useful for answering the user's current query.
<PAST_CONVERSATIONS>
Time: 2025-10-31T10:26:25.254356+00:00
user: I got my niece a red bike for her third birthday.
Time: 2025-10-31T10:25:54.382024+00:00
user: I have a three-year-old niece.
</PAST_CONVERSATIONS>

-----------------------------------------------------------
Config:
{'labels': {'adk_agent_name': 'Generic_QA_Agent'}}
-----------------------------------------------------------
Contents:
{"parts":[{"text":"Can you remind me what I got my niece for her christmas?"}],"role":"user"

Agent Response:  I don't have a record of what you got your niece for Christmas. However, I do remember that you got her a red bike for her third birthday.


### adk.LoadMemoryTool

Now, let's use `LoadMemoryTool`. Unlike `PreloadMemoryTool`, this tool acts like a standard tool. Your agent needs to decide whether the tool should be invoked.

In [77]:
from google import adk


def log_tool_call(tool, args, tool_response, **kwargs):
    """A callback to print the LLM request."""
    print(f"\n*Tool*:\n{tool}")
    print(f"\n*Tool call*:\n{args}")
    print(f"\n*Tool response*:\n{tool_response}\n*********\n")


agent = LlmAgent(
    model=MODEL,
    name="Generic_QA_Agent",
    instruction="Answer the user's questions",
    tools=[adk.tools.load_memory_tool.LoadMemoryTool()],
    before_model_callback=log_system_instructions,
    after_tool_callback=log_tool_call,
)

runner = Runner(
    agent=agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_bank_service,
)

In [78]:
session = await session_service.create_session(app_name=APP_NAME, user_id=USER_ID)

call_agent(
    "Can you remind me what I got my niece for her christmas?",
    session.id,
    user_id=USER_ID,
)

INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash, backend: GoogleLLMVariant.VERTEX_AI, stream: False
DEBUG:google_adk.google.adk.models.google_llm:
LLM Request:
-----------------------------------------------------------
System Instruction:
Answer the user's questions

You are an agent. Your internal name is "Generic_QA_Agent".


You have memory. You can use it to answer questions. If any questions need
you to look up the memory, you should call load_memory function with a query.

-----------------------------------------------------------
Config:
{'tools': [{}], 'labels': {'adk_agent_name': 'Generic_QA_Agent'}}
-----------------------------------------------------------
Contents:
{"parts":[{"text":"Can you remind me what I got my niece for her christmas?"}],"role":"user"}
-----------------------------------------------------------
Functions:
load_memory: {'query': {'type': <Type.STRING: 'STRING'>}} 
---------------------------------------------


*System Instruction*:
Answer the user's questions

You are an agent. Your internal name is "Generic_QA_Agent".


You have memory. You can use it to answer questions. If any questions need
you to look up the memory, you should call load_memory function with a query.

*********



INFO:google_adk.google.adk.models.google_llm:Response received from the model.
DEBUG:google_adk.google.adk.models.google_llm:
LLM Response:
-----------------------------------------------------------
Text:
None
-----------------------------------------------------------
Function calls:
name: load_memory, args: {'query': 'what I got my niece for christmas'}
-----------------------------------------------------------
Raw response:
{"sdk_http_response":{"headers":{"Content-Type":"application/json; charset=UTF-8","Vary":"Origin, X-Origin, Referer","Content-Encoding":"gzip","Date":"Fri, 31 Oct 2025 10:27:38 GMT","Server":"scaffolding on HTTPServer2","X-XSS-Protection":"0","X-Frame-Options":"SAMEORIGIN","X-Content-Type-Options":"nosniff","Transfer-Encoding":"chunked"}},"candidates":[{"content":{"parts":[{"thought_signature":"Cq8DAePx_14ObmPM1V88ZEwn0TJLgcIFDMjfL-x9W4DrrI9lu4__t2SuLPi7-Fz62f_lSnr9tDsw8Z30CiYFKvr_UCJcTNUQV1K5X4hQdrcZ3YlUA0JJHPbF_PbSBYywTQmBoLwZHEUTZ5LV6gHDQGZjYvk8xhOCOxHzlK8VK


*Tool*:

*Tool call*:
{'query': 'what I got my niece for christmas'}

*Tool response*:
memories=[MemoryEntry(content=Content(
  parts=[
    Part(
      text='I have a three-year-old niece.'
    ),
  ],
  role='user'
), author='user', timestamp='2025-10-31T10:25:54.382024+00:00'), MemoryEntry(content=Content(
  parts=[
    Part(
      text='I got my niece a red bike for her third birthday.'
    ),
  ],
  role='user'
), author='user', timestamp='2025-10-31T10:26:25.254356+00:00')]
*********


*System Instruction*:
Answer the user's questions

You are an agent. Your internal name is "Generic_QA_Agent".


You have memory. You can use it to answer questions. If any questions need
you to look up the memory, you should call load_memory function with a query.

*********



INFO:google_adk.google.adk.models.google_llm:Response received from the model.
DEBUG:google_adk.google.adk.models.google_llm:
LLM Response:
-----------------------------------------------------------
Text:
I couldn't find any information in your memory about what you got your niece for Christmas. I can see that you got her a red bike for her third birthday.
-----------------------------------------------------------
Function calls:

-----------------------------------------------------------
Raw response:
{"sdk_http_response":{"headers":{"Content-Type":"application/json; charset=UTF-8","Vary":"Origin, X-Origin, Referer","Content-Encoding":"gzip","Date":"Fri, 31 Oct 2025 10:27:41 GMT","Server":"scaffolding on HTTPServer2","X-XSS-Protection":"0","X-Frame-Options":"SAMEORIGIN","X-Content-Type-Options":"nosniff","Transfer-Encoding":"chunked"}},"candidates":[{"content":{"parts":[{"thought_signature":"Cv0EAePx_15cSnItJa3sg8090CSxV34BVotu6FE_GA5wMvXKRw4NDszYXd3OGrc33McE5Ah3Ybjq_sFEiaohTKAKwiX

Agent Response:  I couldn't find any information in your memory about what you got your niece for Christmas. I can see that you got her a red bike for her third birthday.


If the agent decides that Memory is not helpful for a given query, the Memory tool will not be invoked.

In [79]:
session = await session_service.create_session(app_name=APP_NAME, user_id=USER_ID)

call_agent("Hi!", session.id, user_id=USER_ID)

INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash, backend: GoogleLLMVariant.VERTEX_AI, stream: False
DEBUG:google_adk.google.adk.models.google_llm:
LLM Request:
-----------------------------------------------------------
System Instruction:
Answer the user's questions

You are an agent. Your internal name is "Generic_QA_Agent".


You have memory. You can use it to answer questions. If any questions need
you to look up the memory, you should call load_memory function with a query.

-----------------------------------------------------------
Config:
{'tools': [{}], 'labels': {'adk_agent_name': 'Generic_QA_Agent'}}
-----------------------------------------------------------
Contents:
{"parts":[{"text":"Hi!"}],"role":"user"}
-----------------------------------------------------------
Functions:
load_memory: {'query': {'type': <Type.STRING: 'STRING'>}} 
-----------------------------------------------------------




*System Instruction*:
Answer the user's questions

You are an agent. Your internal name is "Generic_QA_Agent".


You have memory. You can use it to answer questions. If any questions need
you to look up the memory, you should call load_memory function with a query.

*********



INFO:google_adk.google.adk.models.google_llm:Response received from the model.
DEBUG:google_adk.google.adk.models.google_llm:
LLM Response:
-----------------------------------------------------------
Text:
Hello! How can I help you today?
-----------------------------------------------------------
Function calls:

-----------------------------------------------------------
Raw response:
{"sdk_http_response":{"headers":{"Content-Type":"application/json; charset=UTF-8","Vary":"Origin, X-Origin, Referer","Content-Encoding":"gzip","Date":"Fri, 31 Oct 2025 10:27:48 GMT","Server":"scaffolding on HTTPServer2","X-XSS-Protection":"0","X-Frame-Options":"SAMEORIGIN","X-Content-Type-Options":"nosniff","Transfer-Encoding":"chunked"}},"candidates":[{"content":{"parts":[{"thought_signature":"CqcBAePx_175P6dok_z14JIoQcEqpd2WNfEvhtnZoMW-UjPxDwoSHzKNZLN8rDi0A_FnPDjEVmO4THLJGMKjY72i6H8qt3ZjZrEl0z7sSxNQA53RbzQ99msnRnzJ15MOaxgP_EmhBuALB9U2a9CYmq84GW72CbVYYK7G6hpda53afN5M9qcSnHoOVPDq5sAZC4ARuP_JHOMQn5uANkoK

Agent Response:  Hello! How can I help you today?


### Custom callback to retrieve memories

If you want more control over how memories are retrieved, you can create your own custom callback.

For example, you can use the Agent Engine SDK within the callback to retrieve memories without similarity search. This is lower latency than using similarity search.

You can also customize your `scope` rather than using the default "app_name" and "user_id" scope keys used by ADK's memory service.

In [80]:
def retrieve_memories_callback(callback_context, llm_request):
    user_id = callback_context._invocation_context.user_id

    response = client.agent_engines.memories.retrieve(
        name=agent_engine.api_resource.name,
        # Unlike the ADK Memory Service, this does not use App Name in the scope.
        scope={"user_id": user_id},
    )
    memories = [f"* {memory.memory.fact}" for memory in list(response)]
    if not memories:
        # No memories to add to System Instructions.
        return
    # Append formatted memories to the System Instructions
    llm_request.config.system_instruction += (
        "\nHere is information that you have about the user:\n"
    )
    llm_request.config.system_instruction += "\n".join(memories)


def log_tool_call(tool, args, tool_response, **kwargs):
    """A callback to print the LLM request."""
    print(f"\n*Tool*:\n{tool}")
    print(f"\n*Tool call*:\n{args}")
    print(f"\n*Tool response*:\n{log_tool_call}\n*********\n")


agent = LlmAgent(
    model=MODEL,
    name="Generic_QA_Agent",
    instruction="Answer the user's questions",
    before_model_callback=retrieve_memories_callback,
)

runner = Runner(
    agent=agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_bank_service,
)

Since we're using a different scope than the ADK Memory Service, we don't have access to the prior memories. Memories are isolated by their scope dictionary.

In [81]:
session = await session_service.create_session(app_name=APP_NAME, user_id=USER_ID)

call_agent("Hi!", session.id, user_id=USER_ID)

INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash, backend: GoogleLLMVariant.VERTEX_AI, stream: False
DEBUG:google_adk.google.adk.models.google_llm:
LLM Request:
-----------------------------------------------------------
System Instruction:
Answer the user's questions

You are an agent. Your internal name is "Generic_QA_Agent".
-----------------------------------------------------------
Config:
{'labels': {'adk_agent_name': 'Generic_QA_Agent'}}
-----------------------------------------------------------
Contents:
{"parts":[{"text":"Hi!"}],"role":"user"}
-----------------------------------------------------------
Functions:

-----------------------------------------------------------

INFO:google_adk.google.adk.models.google_llm:Response received from the model.
DEBUG:google_adk.google.adk.models.google_llm:
LLM Response:
-----------------------------------------------------------
Text:
Hi there! How can I help you today?
------------------------

Agent Response:  Hi there! How can I help you today?


Let's first generate memories for our new scope.

In [82]:
client.agent_engines.memories.generate(
    scope={"user_id": USER_ID},
    name=agent_engine.api_resource.name,
    direct_contents_source={
        "events": [
            {"content": {"role": "user", "parts": [{"text": "I have four nieces!"}]}}
        ]
    },
)

AgentEngineGenerateMemoriesOperation(
  done=True,
  metadata={
    '@type': 'type.googleapis.com/google.cloud.aiplatform.v1beta1.GenerateMemoriesOperationMetadata',
    'genericMetadata': {
      'createTime': '2025-10-31T10:28:17.839578Z',
      'updateTime': '2025-10-31T10:28:19.751692Z'
    }
  },
  name='projects/568929544199/locations/us-central1/reasoningEngines/7795577023350439936/operations/3111852875676909568',
  response=GenerateMemoriesResponse(
    generated_memories=[
      GenerateMemoriesResponseGeneratedMemory(
        action=<GenerateMemoriesResponseGeneratedMemoryAction.CREATED: 'CREATED'>,
        memory=Memory(
          name='projects/568929544199/locations/us-central1/reasoningEngines/7795577023350439936/memories/4171436614863552512'
        )
      ),
    ]
  )
)

In [83]:
call_agent("What information do you know about me?", session.id, user_id=USER_ID)

INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash, backend: GoogleLLMVariant.VERTEX_AI, stream: False
DEBUG:google_adk.google.adk.models.google_llm:
LLM Request:
-----------------------------------------------------------
System Instruction:
Answer the user's questions

You are an agent. Your internal name is "Generic_QA_Agent".
Here is information that you have about the user:
* I have four nieces.
-----------------------------------------------------------
Config:
{'labels': {'adk_agent_name': 'Generic_QA_Agent'}}
-----------------------------------------------------------
Contents:
{"parts":[{"text":"Hi!"}],"role":"user"}
{"parts":[{"text":"Hi there! How can I help you today?"}],"role":"model"}
{"parts":[{"text":"What information do you know about me?"}],"role":"user"}
-----------------------------------------------------------
Functions:

-----------------------------------------------------------

INFO:google_adk.google.adk.models.google_llm

Agent Response:  I know that you have four nieces.


## Customize your Memory Bank's behavior

So far, we've used Memory Bank with it's default settings. Now, let's customize some configurations. The configuration is set when creating or updating your Agent Engine.

### Customizing Memory Extraction

By default, Memory Bank considers memories that fit the following topics to be meaningful:

* **Personal information (`USER_PERSONAL_INFO`)**: Significant personal information about the user, like names, relationships, hobbies, and important dates. For example, "I work at Google" or "My wedding anniversary is on December 31".
* **User preferences (`USER_PREFERENCES`)**: Stated or implied likes, dislikes, preferred styles, or patterns. For example, "I prefer the middle seat."
* **Key conversation events and task outcomes (`KEY_CONVERSATION_DETAILS`)**: Important milestones or conclusions within the dialogue. For example, "I booked plane tickets for a round trip between JFK and SFO. I leave on June 1, 2025 and return on June 7, 2025."
* **Explicit remember / forget instructions (`EXPLICIT_INSTRUCTIONS`)**: Information that the user explicitly asks the agent to remember or forget. For example, if the user says "Remember that I primarily use Python," Memory Bank generates a memory such as "I primarily use Python."

These are managed topics where Memory Bank manages their definition. If you want to use a subset of these topics or use custom topics, you can provide a "CustomizationConfig" to customize what information Memory Bank should consider meaningful to persist.

#### Managed Topics

We're going to update our Memory Bank to only extract user preferences.

This will take affect for all requests to Memory Bank, regardless of whether you use ADK or Agent Engine SDK.

In [84]:
user_preferences_config = {
    "scope_keys": ["user_id"],
    "memory_topics": [
        {"managed_memory_topic": {"managed_topic_enum": "USER_PREFERENCES"}}
    ],
}


client.agent_engines.update(
    name=agent_engine.api_resource.name,
    config={
        "context_spec": {
            "memory_bank_config": {
                "customization_configs": [user_preferences_config],
            }
        }
    },
)

INFO:vertexai_genai.agentengines:View progress and logs at https://console.cloud.google.com/logs/query?project=devfest-2025-day-1.
INFO:vertexai_genai.agentengines:Agent Engine updated. To use it in another session:
INFO:vertexai_genai.agentengines:agent_engine=client.agent_engines.get(name='projects/568929544199/locations/us-central1/reasoningEngines/7795577023350439936')


AgentEngine(api_resource.name='projects/568929544199/locations/us-central1/reasoningEngines/7795577023350439936')

Memories that fit the `USER_PREFERENCES` category will be persisted.

In [85]:
client.agent_engines.memories.generate(
    scope={"user_id": USER_ID},
    name=agent_engine.api_resource.name,
    direct_contents_source={
        "events": [
            {
                "content": {
                    "role": "user",
                    "parts": [{"text": "I prefer the aisle seat"}],
                }
            }
        ]
    },
)

AgentEngineGenerateMemoriesOperation(
  done=True,
  metadata={
    '@type': 'type.googleapis.com/google.cloud.aiplatform.v1beta1.GenerateMemoriesOperationMetadata',
    'genericMetadata': {
      'createTime': '2025-10-31T10:28:48.867528Z',
      'updateTime': '2025-10-31T10:28:51.708784Z'
    }
  },
  name='projects/568929544199/locations/us-central1/reasoningEngines/7795577023350439936/operations/2085032160636436480',
  response=GenerateMemoriesResponse(
    generated_memories=[
      GenerateMemoriesResponseGeneratedMemory(
        action=<GenerateMemoriesResponseGeneratedMemoryAction.CREATED: 'CREATED'>,
        memory=Memory(
          name='projects/568929544199/locations/us-central1/reasoningEngines/7795577023350439936/memories/7571654333528276992'
        )
      ),
    ]
  )
)

Memories that don't fit `USER_PREFERENCES` won't be persisted.

In [86]:
client.agent_engines.memories.generate(
    scope={"user_id": USER_ID},
    name=agent_engine.api_resource.name,
    direct_contents_source={
        "events": [
            {
                "content": {
                    "role": "user",
                    "parts": [{"text": "I'm going to get my niece a red bike"}],
                }
            }
        ]
    },
)

AgentEngineGenerateMemoriesOperation(
  done=True,
  metadata={
    '@type': 'type.googleapis.com/google.cloud.aiplatform.v1beta1.GenerateMemoriesOperationMetadata',
    'genericMetadata': {
      'createTime': '2025-10-31T10:28:54.860293Z',
      'updateTime': '2025-10-31T10:28:55.932392Z'
    }
  },
  name='projects/568929544199/locations/us-central1/reasoningEngines/7795577023350439936/operations/1341938222120304640'
)

#### Custom Topics
Alternatively, you can define your own custom topics where you define your own labels, descriptions, and examples.

In [87]:
from vertexai.types import MemoryBankCustomizationConfig as CustomizationConfig
from vertexai.types import (
    MemoryBankCustomizationConfigGenerateMemoriesExample as GenerateMemoriesExample,
)
from vertexai.types import (
    MemoryBankCustomizationConfigGenerateMemoriesExampleConversationSource as ConversationSource,
)
from vertexai.types import (
    MemoryBankCustomizationConfigGenerateMemoriesExampleConversationSourceEvent as ConversationSourceEvent,
)
from vertexai.types import (
    MemoryBankCustomizationConfigGenerateMemoriesExampleGeneratedMemory as ExampleGeneratedMemory,
)
from vertexai.types import MemoryBankCustomizationConfigMemoryTopic as MemoryTopic
from vertexai.types import (
    MemoryBankCustomizationConfigMemoryTopicCustomMemoryTopic as CustomMemoryTopic,
)

memory_topic = MemoryTopic(
    custom_memory_topic=CustomMemoryTopic(
        label="business_feedback",
        description="""Specific user feedback about their experience at
the coffee shop. This includes opinions on drinks, food, pastries, ambiance,
staff friendliness, service speed, cleanliness, and any suggestions for
improvement.""",
    )
)


example = GenerateMemoriesExample(
    conversation_source=ConversationSource(
        events=[
            ConversationSourceEvent(
                content=Content(
                    role="model",
                    parts=[
                        Part(
                            text="Welcome back to The Daily Grind! We'd love to hear your feedback on your visit."
                        )
                    ],
                )
            ),
            ConversationSourceEvent(
                content=Content(
                    role="user",
                    parts=[
                        Part(
                            text="Hey. The drip coffee was a bit lukewarm today, which was a bummer. Also, the music was way too loud, I could barely hear my friend."
                        )
                    ],
                )
            ),
        ]
    ),
    generated_memories=[
        ExampleGeneratedMemory(
            fact="The user reported that the drip coffee was lukewarm."
        ),
        ExampleGeneratedMemory(
            fact="The user felt the music in the shop was too loud."
        ),
    ],
)

noop_example = GenerateMemoriesExample(
    conversation_source=ConversationSource(
        events=[
            ConversationSourceEvent(
                content=Content(
                    role="model",
                    parts=[
                        Part(
                            text="Welcome back to The Daily Grind! We'd love to hear your feedback on your visit."
                        )
                    ],
                )
            ),
            ConversationSourceEvent(
                content=Content(
                    role="user", parts=[Part(text="Thanks for the coffee!")]
                )
            ),
        ]
    ),
    generated_memories=[],
)

client.agent_engines.update(
    name=agent_engine.api_resource.name,
    config={
        "context_spec": {
            "memory_bank_config": {
                "customization_configs": [
                    CustomizationConfig(
                        memory_topics=[memory_topic],
                        generate_memories_examples=[example, noop_example],
                    )
                ],
            }
        }
    },
)

ModuleNotFoundError: No module named 'vertexai.types'

In [88]:
client.agent_engines.memories.generate(
    scope={"user_id": "123"},
    name=agent_engine.api_resource.name,
    direct_contents_source={
        "events": [
            {
                "content": {
                    "role": "user",
                    "parts": [{"text": "I prefer the aisle seat"}],
                }
            }
        ]
    },
)

AgentEngineGenerateMemoriesOperation(
  done=True,
  metadata={
    '@type': 'type.googleapis.com/google.cloud.aiplatform.v1beta1.GenerateMemoriesOperationMetadata',
    'genericMetadata': {
      'createTime': '2025-10-31T10:29:08.878497Z',
      'updateTime': '2025-10-31T10:29:09.941949Z'
    }
  },
  name='projects/568929544199/locations/us-central1/reasoningEngines/7795577023350439936/operations/2999262884992647168',
  response=GenerateMemoriesResponse(
    generated_memories=[
      GenerateMemoriesResponseGeneratedMemory(
        action=<GenerateMemoriesResponseGeneratedMemoryAction.CREATED: 'CREATED'>,
        memory=Memory(
          name='projects/568929544199/locations/us-central1/reasoningEngines/7795577023350439936/memories/1539645582618918912'
        )
      ),
    ]
  )
)

In [89]:
import vertexai

client.agent_engines.memories.generate(
    scope={"user_id": "123"},
    name=agent_engine.api_resource.name,
    direct_contents_source={
        "events": [
            {
                "content": {
                    "role": "user",
                    "parts": [{"text": "You should have almond milk"}],
                }
            }
        ]
    },
)

AgentEngineGenerateMemoriesOperation(
  done=True,
  metadata={
    '@type': 'type.googleapis.com/google.cloud.aiplatform.v1beta1.GenerateMemoriesOperationMetadata',
    'genericMetadata': {
      'createTime': '2025-10-31T10:29:13.615751Z',
      'updateTime': '2025-10-31T10:29:17.293037Z'
    }
  },
  name='projects/568929544199/locations/us-central1/reasoningEngines/7795577023350439936/operations/4152184389599494144',
  response=GenerateMemoriesResponse(
    generated_memories=[
      GenerateMemoriesResponseGeneratedMemory(
        action=<GenerateMemoriesResponseGeneratedMemoryAction.CREATED: 'CREATED'>,
        memory=Memory(
          name='projects/568929544199/locations/us-central1/reasoningEngines/7795577023350439936/memories/1991694395216232448'
        )
      ),
    ]
  )
)

In [90]:
list(
    client.agent_engines.memories.retrieve(
        name=agent_engine.api_resource.name, scope={"user_id": "123"}
    )
)

[RetrieveMemoriesResponseRetrievedMemory(
   memory=Memory(
     create_time=datetime.datetime(2025, 10, 31, 10, 29, 9, 827027, tzinfo=TzInfo(UTC)),
     fact='I prefer the aisle seat.',
     name='projects/568929544199/locations/us-central1/reasoningEngines/7795577023350439936/memories/1539645582618918912',
     scope={
       'user_id': '123'
     },
     update_time=datetime.datetime(2025, 10, 31, 10, 29, 9, 827027, tzinfo=TzInfo(UTC))
   )
 ),
 RetrieveMemoriesResponseRetrievedMemory(
   memory=Memory(
     create_time=datetime.datetime(2025, 10, 31, 10, 29, 17, 218651, tzinfo=TzInfo(UTC)),
     fact='I think you should have almond milk.',
     name='projects/568929544199/locations/us-central1/reasoningEngines/7795577023350439936/memories/1991694395216232448',
     scope={
       'user_id': '123'
     },
     topics=[
       MemoryTopicId(
         managed_memory_topic=<ManagedTopicEnum.USER_PREFERENCES: 'USER_PREFERENCES'>
       ),
     ],
     update_time=datetime.datetime(2025,

### Dynamic TTL

You can also configure Memory Bank to automatically set the TTL for any generated or manually created (or updated) memories.

#### Default TTL

Default TTL applies to all operations that create or update a Memory.

In [91]:
client.agent_engines.update(
    name=agent_engine.api_resource.name,
    config={
        "context_spec": {
            "memory_bank_config": {
                "ttl_config": {
                    # 30 days
                    "default_ttl": f"{60 * 60 * 24 * 30}s"
                }
            }
        }
    },
)

INFO:vertexai_genai.agentengines:View progress and logs at https://console.cloud.google.com/logs/query?project=devfest-2025-day-1.
INFO:vertexai_genai.agentengines:Agent Engine updated. To use it in another session:
INFO:vertexai_genai.agentengines:agent_engine=client.agent_engines.get(name='projects/568929544199/locations/us-central1/reasoningEngines/7795577023350439936')


AgentEngine(api_resource.name='projects/568929544199/locations/us-central1/reasoningEngines/7795577023350439936')

In [92]:
response = client.agent_engines.memories.generate(
    scope={"user_id": USER_ID},
    name=agent_engine.api_resource.name,
    direct_contents_source={
        "events": [
            {"content": {"role": "user", "parts": [{"text": "I have four nieces!"}]}}
        ]
    },
    config={"wait_for_completion": True},
)

client.agent_engines.memories.get(
    name=response.response.generated_memories[0].memory.name
)

Memory(
  create_time=datetime.datetime(2025, 10, 31, 10, 28, 18, 981837, tzinfo=TzInfo(UTC)),
  expire_time=datetime.datetime(2025, 11, 30, 10, 29, 31, 340649, tzinfo=TzInfo(UTC)),
  fact='I have four nieces.',
  name='projects/568929544199/locations/us-central1/reasoningEngines/7795577023350439936/memories/4171436614863552512',
  scope={
    'user_id': 'My User'
  },
  topics=[
    MemoryTopicId(
      managed_memory_topic=<ManagedTopicEnum.USER_PERSONAL_INFO: 'USER_PERSONAL_INFO'>
    ),
  ],
  update_time=datetime.datetime(2025, 10, 31, 10, 29, 31, 343156, tzinfo=TzInfo(UTC))
)

#### Granular TTL

The TTL is calculated based on which operation created or updated the Memory. If not set for a given operation, then the operation won't update the Memory's expiration time.

Parameters include:
 * `create_ttl`: Applies to Memories created with CreateMemory operations
 * `generate_created_ttl`: Applies to Memories created with GenerateMemories
 * `generate_updated_ttl`: Applies to Memories updated with GenerateMemories

In [93]:
client.agent_engines.update(
    name=agent_engine.api_resource.name,
    config={
        "context_spec": {
            "memory_bank_config": {
                "ttl_config": {
                    "granular_ttl_config": {
                        # 365 days
                        "generate_created_ttl": f"{24 * 60 * 60 * 365}s",
                    }
                }
            }
        }
    },
)

INFO:vertexai_genai.agentengines:View progress and logs at https://console.cloud.google.com/logs/query?project=devfest-2025-day-1.
INFO:vertexai_genai.agentengines:Agent Engine updated. To use it in another session:
INFO:vertexai_genai.agentengines:agent_engine=client.agent_engines.get(name='projects/568929544199/locations/us-central1/reasoningEngines/7795577023350439936')


AgentEngine(api_resource.name='projects/568929544199/locations/us-central1/reasoningEngines/7795577023350439936')

In [94]:
response = client.agent_engines.memories.generate(
    scope={"user_id": USER_ID},
    name=agent_engine.api_resource.name,
    direct_contents_source={
        "events": [
            {
                "content": {
                    "role": "user",
                    "parts": [{"text": "Remember that I work at Zindua School!"}],
                }
            }
        ]
    },
)

client.agent_engines.memories.get(
    name=response.response.generated_memories[0].memory.name
)

Memory(
  create_time=datetime.datetime(2025, 10, 31, 10, 29, 42, 155270, tzinfo=TzInfo(UTC)),
  expire_time=datetime.datetime(2026, 10, 31, 10, 29, 42, 142526, tzinfo=TzInfo(UTC)),
  fact='I work at Zindua School.',
  name='projects/568929544199/locations/us-central1/reasoningEngines/7795577023350439936/memories/3014574460582756352',
  scope={
    'user_id': 'My User'
  },
  update_time=datetime.datetime(2025, 10, 31, 10, 29, 42, 155270, tzinfo=TzInfo(UTC))
)

In [95]:
response = client.agent_engines.memories.generate(
    scope={"user_id": USER_ID},
    name=agent_engine.api_resource.name,
    direct_contents_source={
        "events": [
            {"content": {"role": "user", "parts": [{"text": "I work at Google!"}]}}
        ]
    },
)

client.agent_engines.memories.get(
    name=response.response.generated_memories[0].memory.name
)

Memory(
  create_time=datetime.datetime(2025, 10, 31, 10, 29, 54, 10722, tzinfo=TzInfo(UTC)),
  expire_time=datetime.datetime(2026, 10, 31, 10, 29, 54, 2456, tzinfo=TzInfo(UTC)),
  fact='I work at Google.',
  name='projects/568929544199/locations/us-central1/reasoningEngines/7795577023350439936/memories/7283423957376565248',
  scope={
    'user_id': 'My User'
  },
  topics=[
    MemoryTopicId(
      managed_memory_topic=<ManagedTopicEnum.USER_PERSONAL_INFO: 'USER_PERSONAL_INFO'>
    ),
  ],
  update_time=datetime.datetime(2025, 10, 31, 10, 29, 54, 10722, tzinfo=TzInfo(UTC))
)

## Cleaning up

It's always a best practice in cloud development to clean up resources you no longer need to avoid incurring unexpected costs. This final cell deletes the AgentEngine resources we created throughout this tutorial.

In [50]:
delete_agent_engines = True

agent_engine_name = agent_engine.api_resource.name

if delete_agent_engines:
    # Delete agent engine
    client.agent_engines.delete(name=agent_engine_name, force=True)

NameError: name 'false' is not defined